# Day 09. Exercise 03
# Ensembles

## 0. Imports

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.ensemble import VotingClassifier, BaggingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from joblib import dump

## 1. Preprocessing

1. Create the same dataframe as in the previous exercise.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test` and then get `X_train`, `y_train`, `X_valid`, `y_valid` from the previous `X_train`, `y_train`. Use the additional parameter `stratify`.

In [142]:
df = pd.read_csv("../data/day-of-week-not-scaled.csv")
df["dayofweek"] = pd.read_csv("../data/dayofweek.csv", usecols=["dayofweek"])

x = df.drop(columns="dayofweek")
y = df["dayofweek"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=21, stratify=y)

## 2. Individual classifiers

1. Train SVM, decision tree and random forest again with the best parameters that you got from the 01 exercise with `random_state=21` for all of them.
2. Evaluate `accuracy`, `precision`, and `recall` for them on the validation set.
3. The result of each cell of the section should look like this:

```
accuracy is 0.87778
precision is 0.88162
recall is 0.87778
```

In [143]:
model_svc = SVC(random_state=21, C=10, class_weight=None, gamma="auto", kernel="rbf", probability=True)
model_svc.fit(x_train, y_train)
predict = model_svc.predict(x_test)
print(f"accuracy is {accuracy_score(y_test, predict):.5f}")
print(f"precision is {precision_score(y_test, predict, average='weighted'):.5f}")
print(f"recall is {recall_score(y_test, predict, average='weighted'):.5f}")

accuracy is 0.88757
precision is 0.89267
recall is 0.88757


In [144]:
model_tree = DecisionTreeClassifier(random_state=21, class_weight="balanced", criterion="gini", max_depth=21)
model_tree.fit(x_train, y_train)
predict = model_tree.predict(x_test)
print(f"accuracy is {accuracy_score(y_test, predict):.5f}")
print(f"precision is {precision_score(y_test, predict, average='weighted'):.5f}")
print(f"recall is {recall_score(y_test, predict, average='weighted'):.5f}")

accuracy is 0.88462
precision is 0.88765
recall is 0.88462


In [145]:
model_forest = RandomForestClassifier(random_state=21, class_weight="balanced", criterion="entropy", max_depth=24, n_estimators=100)
model_forest.fit(x_train, y_train)
predict = model_forest.predict(x_test)
print(f"accuracy is {accuracy_score(y_test, predict):.5f}")
print(f"precision is {precision_score(y_test, predict, average='weighted'):.5f}")
print(f"recall is {recall_score(y_test, predict, average='weighted'):.5f}")

accuracy is 0.92604
precision is 0.92754
recall is 0.92604


## 3. Voting classifiers

1. Using `VotingClassifier` and the three models that you have just trained, calculate the `accuracy`, `precision`, and `recall` on the validation set.
2. Play with the other parameteres.
3. Calculate the `accuracy`, `precision` and `recall` on the test set for the model with the best weights in terms of accuracy (if there are several of them with equal values, choose the one with the higher precision).

In [146]:
model_svc = SVC(random_state=21, C=10, class_weight=None, gamma="auto", kernel="rbf", probability=True)
model_tree = DecisionTreeClassifier(random_state=21, class_weight="balanced", criterion="gini", max_depth=21)
model_forest = RandomForestClassifier(random_state=21, class_weight="balanced", criterion="entropy", max_depth=24, n_estimators=100)

In [147]:
model_voting = VotingClassifier(estimators=[("svc", model_svc), ("tree", model_tree), ("forest", model_forest)])
model_voting.fit(x_train, y_train)
predict = model_voting.predict(x_test)
print(f"accuracy is {accuracy_score(y_test, predict):.5f}")
print(f"precision is {precision_score(y_test, predict, average='weighted'):.5f}")
print(f"recall is {recall_score(y_test, predict, average='weighted'):.5f}")

accuracy is 0.91420
precision is 0.91538
recall is 0.91420


In [148]:
param_grid = {
    "voting": ["soft", "hard"],
    "weights" : [[1,1,1], [1,2,3], [3,2,1]]
}
grid_search = GridSearchCV(estimator=model_voting, param_grid=param_grid, scoring="accuracy", n_jobs=-1)
grid_search.fit(x_train, y_train)
grid_search.best_params_

{'voting': 'hard', 'weights': [1, 2, 3]}

In [149]:
model_voting = VotingClassifier(estimators=[("svc", model_svc), ("tree", model_tree), ("forest", model_forest)], voting="hard", weights=[1,2,3])
model_voting.fit(x_train, y_train)
predict = model_voting.predict(x_test)
print(f"accuracy is {accuracy_score(y_test, predict):.5f}")
print(f"precision is {precision_score(y_test, predict, average='weighted'):.5f}")
print(f"recall is {recall_score(y_test, predict, average='weighted'):.5f}")

accuracy is 0.92308
precision is 0.92491
recall is 0.92308


## 4. Bagging classifiers

1. Using `BaggingClassifier` and `SVM` with the best parameters create an ensemble, try different values of the `n_estimators`, use `random_state=21`.
2. Play with the other parameters.
3. Calculate the `accuracy`, `precision`, and `recall` for the model with the best parameters (in terms of accuracy) on the test set (if there are several of them with equal values, choose the one with the higher precision)

In [150]:
model_svc = SVC(random_state=21, C=10, class_weight=None, gamma="auto", kernel="rbf", probability=True)
model_Bagging = BaggingClassifier(base_estimator=model_svc, random_state=21)
model_Bagging.fit(x_train, y_train)
predict = model_Bagging.predict(x_test)
print(f"accuracy is {accuracy_score(y_test, predict):.5f}")
print(f"precision is {precision_score(y_test, predict, average='weighted', zero_division=0):.5f}")
print(f"recall is {recall_score(y_test, predict, average='weighted'):.5f}")

accuracy is 0.88757
precision is 0.89182
recall is 0.88757


In [151]:
model_svc = SVC(random_state=21, C=10, class_weight=None, gamma="auto", kernel="rbf", probability=True)
model_Bagging = BaggingClassifier(base_estimator=model_svc, random_state=21, n_estimators=10)
model_Bagging.fit(x_train, y_train)
predict = model_Bagging.predict(x_test)
print(f"accuracy is {accuracy_score(y_test, predict):.5f}")
print(f"precision is {precision_score(y_test, predict, average='weighted', zero_division=0):.5f}")
print(f"recall is {recall_score(y_test, predict, average='weighted'):.5f}")

accuracy is 0.88757
precision is 0.89182
recall is 0.88757


In [152]:
model_svc = SVC(random_state=21, C=10, class_weight=None, gamma="auto", kernel="rbf", probability=True)
model_Bagging = BaggingClassifier(base_estimator=model_svc, random_state=21, n_estimators=1)
model_Bagging.fit(x_train, y_train)
predict = model_Bagging.predict(x_test)
print(f"accuracy is {accuracy_score(y_test, predict):.5f}")
print(f"precision is {precision_score(y_test, predict, average='weighted', zero_division=0):.5f}")
print(f"recall is {recall_score(y_test, predict, average='weighted'):.5f}")

accuracy is 0.83432
precision is 0.84210
recall is 0.83432


In [164]:
model_svc = SVC(random_state=21, C=10, class_weight=None, gamma="auto", kernel="rbf", probability=True)
model_Bagging = BaggingClassifier(base_estimator=model_svc, random_state=21, n_estimators=50)
model_Bagging.fit(x_train, y_train)
predict = model_Bagging.predict(x_test)
print(f"accuracy is {accuracy_score(y_test, predict):.5f}")
print(f"precision is {precision_score(y_test, predict, average='weighted', zero_division=0):.5f}")
print(f"recall is {recall_score(y_test, predict, average='weighted'):.5f}")

accuracy is 0.90828
precision is 0.91091
recall is 0.90828


## 5. Stacking classifiers

1. To achieve reproducibility in this case you will have to create an object of cross-validation generator: `StratifiedKFold(n_splits=n, shuffle=True, random_state=21)`, where `n` you will try to optimize (the details are below).
2. Using `StackingClassifier` and the three models that you have recently trained, calculate the `accuracy`, `precision` and `recall` on the validation set, try different values of `n_splits` `[2, 3, 4, 5, 6, 7]` in the cross-validation generator and parameter `passthrough` in the classifier itself,
3. Calculate the `accuracy`, `precision`, and `recall` for the model with the best parameters (in terms of accuracy) on the test set (if there are several of them with equal values, choose the one with the higher precision). Use `final_estimator=LogisticRegression(solver='liblinear')`.

In [154]:
model_svc = SVC(random_state=21, C=10, class_weight=None, gamma="auto", kernel="rbf", probability=True)
model_tree = DecisionTreeClassifier(random_state=21, class_weight="balanced", criterion="gini", max_depth=21)
model_forest = RandomForestClassifier(random_state=21, class_weight="balanced", criterion="entropy", max_depth=24, n_estimators=100)

n_splits = [2, 3, 4, 5, 6, 7]
best_accuracy = 0
best_precision = 0
best_recall = 0
best_model = None

for n in n_splits:
        for passthrough in [True, False]:
                kf = StratifiedKFold(n_splits=n, shuffle=True, random_state=21)
                for train_index, test_index in kf.split(x_train, y_train):
                        x_train_kf, x_test_kf = x_train.iloc[train_index], x_train.iloc[test_index]
                        y_train_kf, y_test_kf = y_train.iloc[train_index], y_train.iloc[test_index]

                        model_stacking = StackingClassifier(estimators=[("svc", model_svc), ("tree", model_tree), ("forest", model_forest)], 
                                                        final_estimator=LogisticRegression(solver="liblinear"),
                                                        passthrough=passthrough)
                        model_stacking.fit(x_train_kf, y_train_kf)
                        predict = model_stacking.predict(x_test_kf)

                        accuracy = accuracy_score(y_test_kf, predict)
                        precision = precision_score(y_test_kf, predict, average='weighted', zero_division=1)
                        recall = recall_score(y_test_kf, predict, average='weighted', zero_division=1)

                        if accuracy > best_accuracy or (accuracy == best_accuracy and precision > best_precision):
                                best_accuracy = accuracy
                                best_precision = precision
                                best_recall = recall
                                best_model = model_stacking

predict = best_model.predict(x_test)
print(f"accuracy is {accuracy_score(y_test, predict):.5f}")
print(f"precision is {precision_score(y_test, predict, average='weighted', zero_division=0):.5f}")
print(f"recall is {recall_score(y_test, predict, average='weighted'):.5f}")

accuracy is 0.92308
precision is 0.92330
recall is 0.92308


## 6. Predictions

1. Choose the best model in terms of accuracy (if there are several of them with equal values, choose the one with the higher precision).
2. Analyze: for which weekday your model makes the most errors (in % of the total number of samples of that class in your full dataset), for which labname and for which users.
3. Save the model.

In [163]:
predict = best_model.predict(x)
df["predict"] = predict
error_perc_by_day = df[df.predict != df.dayofweek].dayofweek.value_counts() / len(predict)
max_day = error_perc_by_day.idxmax()
max_error = error_perc_by_day.max() * 100
print(f"max day error: {max_day}, error percent: {max_error:.3f}%")

max day error: 5, error percent: 0.712%


In [160]:
error = df[df["predict"] != df["dayofweek"]]
users = [i for i in df.columns if i.startswith("uid_")]
labnames = [i for i in df.columns if i.startswith("labname_")]

In [161]:
max_error = 0
max_user = ''
for user in users:
    error_perc = error[user].sum() / len(predict)
    if error_perc > max_error:
        max_error = error_perc
        max_user = user
print(f"max user error: {max_user}, error percent: {(max_error * 100):.3f}%")

max user error: uid_user_13, error percent: 0.237%


In [162]:
max_error = 0
max_user = ''
for lab in labnames:
    error_perc = error[lab].sum() / len(predict)
    if error_perc > max_error:
        max_error = error_perc
        max_lab = lab
print(f"max lab error: {max_lab}, error percent: {(max_error * 100):.3f}%")

max lab error: labname_project1, error percent: 1.127%


In [169]:
joblib.dump(best_model, "best_model.joblib")

['best_model.joblib']